In [10]:
import pandas as pd
import csv
import pyltr
import numpy as np
import datetime
import re
import time
import dawg

suffixes = pd.read_csv("../data/Freq_background.csv", index_col='Unnamed: 0')
with open('../data/sorted_popular_queries.csv', mode='r') as infile:
    reader = csv.reader(infile)
    skipheader = next(reader)
    data=[tuple([line[1], int(line[2])]) for line in reader]
sortedpopulardict = dawg.IntCompletionDAWG(data)

with open('../data/ngram_dict.csv', mode='r') as infile:
    reader = csv.reader(infile)
    ngram_dict = {rows[0]:rows[1] for rows in reader}
prefix_suffix_pairs_background = pd.read_csv("../data/prefix_suffix_pairs.txt")


In [11]:
top100k = suffixes.iloc[range(100000),:]
suffixes = []
data = []

In [12]:
from nltk import ngrams
def get_single_ngram_features(query_input, ngram_dict):
    ngram_scores = []

    for n in range(1,7):
        sixgrams = ngrams(query_input.split(), n)
        ngram_score = 0
        for grams in sixgrams:
            try:
                ngram_score = ngram_score + float(ngram_dict[' '.join(grams)])
            except:
                pass
        ngram_scores.append(ngram_score)
    return ngram_scores

def get_ngram_features(inputs, ngram_dict):
    results = [[get_single_ngram_features(x, ngram_dict)] for x in inputs];
    return results


In [13]:
prefix_suffix_pairs_background = []
removed_session_training = pd.read_csv('../data/removed_session_training.csv', header=None, index_col=[0])
removed_session_training[1] = removed_session_training[1].str.replace('[\W_]+', ' ')

In [14]:
training_queries = removed_session_training[1:100].reset_index(drop=True)

In [56]:
candidate_prefixes = []
right_query = []

for j in range(len(training_queries)):
    current_query = str(training_queries[1][j])
    split_query = current_query.split(" ")
    suffix = ' '.join(split_query[1:])
    prefix = split_query[0] + " "

    for i in range(len(suffix)+1):
        candidate_prefixes.append(prefix + suffix[:i])
        right_query.append(current_query)
    
print("done")

done


In [57]:
candidate_prefixes

['loislaw ',
 'loislaw c',
 'loislaw co',
 'loislaw com',
 'www ',
 'www t',
 'www ta',
 'www tab',
 'www tabi',
 'www tabie',
 'www tabiec',
 'www tabiecu',
 'www tabiecum',
 'www tabiecumm',
 'www tabiecummi',
 'www tabiecummin',
 'www tabiecumming',
 'www tabiecummings',
 'www tabiecummings ',
 'www tabiecummings c',
 'www tabiecummings co',
 'www tabiecummings com',
 'idx ',
 'idx t',
 'idx te',
 'idx tec',
 'idx tech',
 'idx techs',
 'idx techso',
 'idx techsol',
 'idx techsols',
 'idx techsolsc',
 'idx techsolsc ',
 'idx techsolsc c',
 'idx techsolsc co',
 'idx techsolsc com',
 'www ',
 'www b',
 'www br',
 'www bri',
 'www brid',
 'www bridl',
 'www bridle',
 'www bridlea',
 'www bridlean',
 'www bridleand',
 'www bridleandb',
 'www bridleandbi',
 'www bridleandbit',
 'www bridleandbit ',
 'www bridleandbit c',
 'www bridleandbit co',
 'www bridleandbit com',
 'gall ',
 'gall s',
 'gall st',
 'gall sto',
 'gall ston',
 'gall stone',
 'gall stones',
 'gallstones ',
 'http ',
 'ht

In [58]:
all_candidates = np.array([])
relevant_candidate = np.array([])
qids = np.array([])
ngram_features = np.array([])


for candidate_prefix_i in range(len(candidate_prefixes)):
    print(str(candidate_prefix_i) + '/' + str(len(candidate_prefixes)))
    candidate_prefix = candidate_prefixes[candidate_prefix_i]
    original_query = right_query[candidate_prefix_i]
    
    splitted = re.split('[\W_]+',candidate_prefix)
    if candidate_prefix[len(candidate_prefix)-1] == ' ':
        endterm = splitted[len(splitted)-2] + " "
    else:
        endterm = splitted[len(splitted)-1]
    
    out = top100k[top100k['0'].str.startswith(endterm)][:10]
    keys_pop = sortedpopulardict.keys(candidate_prefix)
    
    outcombined1 = candidate_prefix[:-len(endterm)] + out['0'].astype(str)
    
    current_candidates = outcombined1
    current_candidates.append(pd.Series(keys_pop))
    
    all_candidates = np.append(all_candidates, current_candidates)
    relevant_candidate = np.append(relevant_candidate, (current_candidates == original_query).apply(float))
    
    qids = np.append(qids,np.ones(len(current_candidates)) * candidate_prefix_i)

    ngram_features = np.append(ngram_features, get_ngram_features(current_candidates, ngram_dict))

0/1276
1/1276
2/1276
3/1276
4/1276
5/1276
6/1276
7/1276
8/1276
9/1276
10/1276
11/1276
12/1276
13/1276
14/1276
15/1276
16/1276
17/1276
18/1276
19/1276
20/1276
21/1276
22/1276
23/1276
24/1276
25/1276
26/1276
27/1276
28/1276
29/1276
30/1276
31/1276
32/1276
33/1276
34/1276
35/1276
36/1276
37/1276
38/1276
39/1276
40/1276
41/1276
42/1276
43/1276
44/1276
45/1276
46/1276
47/1276
48/1276
49/1276
50/1276
51/1276
52/1276
53/1276
54/1276
55/1276
56/1276
57/1276
58/1276
59/1276
60/1276
61/1276
62/1276
63/1276
64/1276
65/1276
66/1276
67/1276
68/1276
69/1276
70/1276
71/1276
72/1276
73/1276
74/1276
75/1276
76/1276
77/1276
78/1276
79/1276
80/1276
81/1276
82/1276
83/1276
84/1276
85/1276
86/1276
87/1276
88/1276
89/1276
90/1276
91/1276
92/1276
93/1276
94/1276
95/1276
96/1276
97/1276
98/1276
99/1276
100/1276
101/1276
102/1276
103/1276
104/1276
105/1276
106/1276
107/1276
108/1276
109/1276
110/1276
111/1276
112/1276
113/1276
114/1276
115/1276
116/1276
117/1276
118/1276
119/1276
120/1276
121/1276
122/1276
123

923/1276
924/1276
925/1276
926/1276
927/1276
928/1276
929/1276
930/1276
931/1276
932/1276
933/1276
934/1276
935/1276
936/1276
937/1276
938/1276
939/1276
940/1276
941/1276
942/1276
943/1276
944/1276
945/1276
946/1276
947/1276
948/1276
949/1276
950/1276
951/1276
952/1276
953/1276
954/1276
955/1276
956/1276
957/1276
958/1276
959/1276
960/1276
961/1276
962/1276
963/1276
964/1276
965/1276
966/1276
967/1276
968/1276
969/1276
970/1276
971/1276
972/1276
973/1276
974/1276
975/1276
976/1276
977/1276
978/1276
979/1276
980/1276
981/1276
982/1276
983/1276
984/1276
985/1276
986/1276
987/1276
988/1276
989/1276
990/1276
991/1276
992/1276
993/1276
994/1276
995/1276
996/1276
997/1276
998/1276
999/1276
1000/1276
1001/1276
1002/1276
1003/1276
1004/1276
1005/1276
1006/1276
1007/1276
1008/1276
1009/1276
1010/1276
1011/1276
1012/1276
1013/1276
1014/1276
1015/1276
1016/1276
1017/1276
1018/1276
1019/1276
1020/1276
1021/1276
1022/1276
1023/1276
1024/1276
1025/1276
1026/1276
1027/1276
1028/1276
1029/1276
1030/12

In [59]:
ngram_features

array([17.78639002, 15.88141004,  0.        , ...,  0.        ,
        0.        ,  0.        ])

In [278]:
# query = input("Query: ")
# original_query = input("Original query: ")

# search_query = re.sub('[\W_]+',' ', query)
# original_query = re.sub('[\W_]+',' ', original_query)

# startt = time.time()
# splitted = re.split('[\W_]+',search_query)
# print(splitted)
# if query[len(query)-1] == ' ':
#     endterm = splitted[len(splitted)-2] + " "
# else:
#     endterm = splitted[len(splitted)-1]

# print("Endterm -> '" + endterm + "'")

# out = top100k[top100k['0'].str.startswith(endterm)].nlargest(10, 'counts')
# out2 = sortedpopular[sortedpopular['0'].str.startswith(search_query)].nlargest(40, 'counts')

# outcombined1 = query[:-len(endterm)] + out['0'].astype(str)
# time.time() - startt
# all_candidates = outcombined1.append(out2.iloc[:,0])
# relevant_candidate = (all_candidates == original_query).apply(float)

# train_input = pd.DataFrame({
#     'query': all_candidates,
#     'relevant': relevant_candidate,
#     'qid': 1
# })
  
# train_input['ngram_features'] = get_ngram_features(all_candidates, ngram_dict)
# print(train_input)


In [90]:
metric = pyltr.metrics.err.ERR(highest_score=1)

model = pyltr.models.LambdaMART(
    metric=metric,
    verbose=1,
)

TX2 = ngram_features.reshape(-1, 6) 
Ty2 = np.array(relevant_candidate)
Tqids2 = np.array(qids)
model.fit(TX2, Ty2, Tqids2)

 Iter  Train score    Remaining                           Monitor Output 
    1       0.0846       14.79s                                         
    2       0.0878       15.10s                                         
    3       0.0908       14.51s                                         
    4       0.0905       14.28s                                         
    5       0.0905       14.12s                                         
    6       0.0905       13.91s                                         
    7       0.0950       13.73s                                         
    8       0.0973       13.63s                                         
    9       0.1039       13.47s                                         
   10       0.1102       13.34s                                         
   15       0.1387       12.41s                                         
   20       0.1575       11.57s                                         
   25       0.1605       10.84s                   

In [295]:
with open('../data/train.txt') as trainfile, \
        open('../data/vali.txt') as valifile, \
        open('../data/test.txt') as evalfile:
    TX, Ty, Tqids, _ = pyltr.data.letor.read_dataset(trainfile)
    VX, Vy, Vqids, _ = pyltr.data.letor.read_dataset(valifile)
    EX, Ey, Eqids, _ = pyltr.data.letor.read_dataset(evalfile)

In [85]:
EX = TX2 
Ey = Ty2
Eqids = Tqids2

Epred = model.predict(EX)

In [86]:
print('Random ranking:' + str(metric.calc_mean_random(Eqids, Ey)))
print('Our model:' + str(metric.calc_mean(Eqids, Ey, Epred)))

Random ranking:0.08518135569395788
Our model:0.19342048354309932


In [87]:
inputs = ['hello kitty', 'hello my name is jeff', 'hello i am martijn', 'hello who are you', 'hello goodbye']
EX = get_ngram_features(inputs,ngram_dict)
EX = np.append([], EX)
EX = EX.reshape(-1, 6) 
Ey = np.array([1, 0, 0, 0, 0])
Eqids = np.ones(4)

Epred = model.predict(EX)
print(Epred)
print('Random ranking:' + str(metric.calc_mean_random(Eqids, Ey)))
print('Our model:' + str(metric.calc_mean(Eqids, Ey, Epred)))
output = pd.DataFrame({
    'inputs': inputs,
    'value': Epred
})
output.sort_values('value', ascending=False)

[-0.87099986 -1.91213678 -2.83402747 -1.88916165  1.34656631]
Random ranking:0.27208333333333334
Our model:0.5


,inputs,value
4,hello goodbye,1.346566
0,hello kitty,-0.871000
3,hello who are you,-1.889162
1,hello my name is jeff,-1.912137
2,hello i am martijn,-2.834027


,inputs,value
4,hello goodbye,1.473753
0,hello kitty,-1.210034
1,hello my name is jeff,-1.655306
3,hello who are you,-1.769709
2,hello i am martijn,-2.874387
